In [1]:
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

In [3]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100.0%


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100.0%


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100.0%


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100.0%

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break   

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [55]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 2048),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(256, 10),
            nn.ReLU()
        )

    def forward(self, x: torch.Tensor):
        x = self.flatten(x)
        return self.linear_relu_stack(x)

cuda


In [56]:
model: NeuralNetwork = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): ReLU()
    (5): Linear(in_features=1024, out_features=512, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.3, inplace=False)
    (11): Linear(in_features=256, out_features=10, bias=True)
    (12): ReLU()
  )
)


In [27]:
model.load_state_dict(torch.load("model3.pth"))

RuntimeError: Error(s) in loading state_dict for NeuralNetwork:
	Missing key(s) in state_dict: "linear_relu_stack.3.weight", "linear_relu_stack.3.bias", "linear_relu_stack.5.weight", "linear_relu_stack.5.bias", "linear_relu_stack.11.weight", "linear_relu_stack.11.bias". 
	Unexpected key(s) in state_dict: "linear_relu_stack.2.weight", "linear_relu_stack.2.bias", "linear_relu_stack.4.weight", "linear_relu_stack.4.bias", "linear_relu_stack.6.weight", "linear_relu_stack.6.bias". 
	size mismatch for linear_relu_stack.8.weight: copying a param with shape torch.Size([10, 256]) from checkpoint, the shape in current model is torch.Size([256, 512]).
	size mismatch for linear_relu_stack.8.bias: copying a param with shape torch.Size([10]) from checkpoint, the shape in current model is torch.Size([256]).

In [57]:
loss_function = nn.CrossEntropyLoss()
optimzer = torch.optim.SGD(model.parameters(), lr=1e-5, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimzer, step_size=5, gamma=0.99)

In [58]:
def train(dataloader: DataLoader, model: NeuralNetwork, loss_function, optimizer: torch.optim.Optimizer):
    model.train()
    m = len(dataloader.dataset)
    epoch_cost = 0
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = torch.squeeze(X).to(device), y.to(device)
        

        pred = model(X) # calculate predictions
        loss = loss_function(pred, y) # calculate loss
        epoch_cost += loss.item()

        loss.backward() # backpropagation
        optimizer.step() # update parameters 

        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{m:>5d}]")


In [59]:
def test(dataloader: DataLoader, model: NeuralNetwork, loss_function):
    model.eval()
    m = len(dataloader.dataset)
    batch_count = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad(): # beräkna inte gradienter i onödan
        for X, y in dataloader:
            X, y = X.to(device), y.to(device) # flytta data till device t.ex. cuda dvs grafikkortet
            pred = model(X)
            test_loss += loss_function(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss = test_loss / batch_count
    correct = correct / m
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [60]:
epochs = 50

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimzer)
    test(test_dataloader, model, loss_function)
    print("Learning rate: ", optimzer.param_groups[0]['lr'])
    scheduler.step()

Epoch 1
-------------------------------
loss: 2.300208 [    0/60000]
loss: 2.305763 [ 3200/60000]
loss: 2.304620 [ 6400/60000]
loss: 2.301605 [ 9600/60000]
loss: 2.297458 [12800/60000]
loss: 2.299821 [16000/60000]
loss: 2.308565 [19200/60000]
loss: 2.303576 [22400/60000]
loss: 2.302880 [25600/60000]
loss: 2.296221 [28800/60000]
loss: 2.302065 [32000/60000]
loss: 2.298431 [35200/60000]
loss: 2.293895 [38400/60000]
loss: 2.300307 [41600/60000]
loss: 2.305403 [44800/60000]
loss: 2.299619 [48000/60000]
loss: 2.301297 [51200/60000]
loss: 2.301133 [54400/60000]
loss: 2.295105 [57600/60000]
Test Error: 
 Accuracy: 11.3%, Avg loss: 2.296418 

Learning rate:  1e-05
Epoch 2
-------------------------------
loss: 2.296488 [    0/60000]
loss: 2.303417 [ 3200/60000]
loss: 2.295429 [ 6400/60000]
loss: 2.296817 [ 9600/60000]
loss: 2.291312 [12800/60000]
loss: 2.295807 [16000/60000]
loss: 2.303492 [19200/60000]
loss: 2.293933 [22400/60000]
loss: 2.297136 [25600/60000]
loss: 2.275913 [28800/60000]
loss:

KeyboardInterrupt: 

In [119]:
torch.save(model.state_dict(), "model3.pth")
print("Saved PyTorch Model State to model.pth")
!pwd

Saved PyTorch Model State to model.pth


/c/Users/x603e/code/notebook


In [61]:
def calc_bias(model: NeuralNetwork, train_dataloader: DataLoader, test_dataloader: DataLoader):
    model.eval()
    train_acc, test_acc = 0, 0

    with torch.no_grad():
        for X, y in train_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            train_acc += (pred.argmax(1) == y).type(torch.float).sum().item()
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_acc += (pred.argmax(1) == y).type(torch.float).sum().item()
    train_acc = (train_acc / len(train_dataloader.dataset)) * 100
    test_acc = (test_acc / len(test_dataloader.dataset)) * 100
    #return train_acc, test_acc, abs(train_acc - test_acc)
    print(f"Train Accuracy: {train_acc:>0.3f}%, Test Accuracy: {test_acc:>0.3f}%, Difference/Bias: {abs(train_acc - test_acc):>0.3f}%")


In [62]:
calc_bias(model, train_dataloader, test_dataloader)

Train Accuracy: 98.982%, Test Accuracy: 97.820%, Difference/Bias: 1.162%
